# Local Linear DeePC MT

## Introduction

### Rocket Dynamics, State, and Action Spaces


A free body diagram illustrating the forces acting on the simplified 2D rocket is shown below. Note that we do not consider air resistance or aerodynamic effects in the 2D model, for simplicity.


<img src="https://gitlab.ethz.ch/bsaverio/coco-project/-/raw/main/images/Rocket_Dynamics.png" width=400/>

### Actions
Three inputs are available for steering the rocket:
- $F_E$: the thrust produced by the main engine in Newtons (N), acting directly on the rocket body at the point where the nozzle pivots;
- $F_S$: the thrust produced by the side gas thrusters in Newtons (N), defined as the difference $F_L - F_R$, acting at a distance $l_2$ from the rocket center of mass;
- $\phi$: the angle of the nozzle with respect to the rocket body in radians (rad), which changes the direction of $F_E$. It can be discontinuous and it has instant response up to the 60 fps frame rate of our model (sampling time of the environment).

The three inputs are provided to the environment as an action:

$$\textbf{u} = \begin{bmatrix}F_E & F_S & \phi\end{bmatrix}$$

which is applied to the environment at the 60 fps frame rate. In other words, your controller must provide 60 actions per second of simulation time. These could be new actions, or the same action could be applied across multiple simulation steps.

Unlike the real world, there is no hard constraint on how long your controller has to produce a next action, although we should be able to run your code in a reasonable amount of time. In our own testing, a well performing optimization-based method took much less than 1 minute to run.

<font color='red'>IMPORTANT</font>: The controller should return *normalized* actions, as a fraction of full scale. The main engine thrust is clipped to $[0, 1]$, while the side engine thrust and nozzle angle is clipped to $[-1, 1]$. The environment then scales these to the appropriate ranges (see *Limits on the Actuators*).


### State Variables
The state of the rocket is defined by the following variables:
- $(x, y)$: the 2D position of the rocket center of mass in meters (m);
- $(\dot{x}, \dot{y})$: the velocity of the rocket in meters per second (m/s);
- $\theta$: the angle of the rocket in radians (rad);
- $\dot{\theta}$: the angular velocity of the rocket in radians per second (rad/s);
- $c_L, c_R$: binary variables indicating whether the left or right legs are in contact with the environment, respectively (equal to 0 otherwise).

The environment returns these variables as an observation:
$$\textbf{x} = \begin{bmatrix}x & y & \dot{x} & \dot{y} & \theta & \dot{\theta}& c_L & c_R\end{bmatrix}$$

The last two variables ($c_L, c_R$) may generally be ignored in your control implementation, but may be useful e.g. for turning off one or more actions once ground contact is made.

<font color='red'>IMPORTANT</font>: All variables are with respect to world frame, which is located in the *bottom-left* corner of the simulation window. The x-axis increases going right (horizontally) along the window and the y-axis increases going up (vertically). Positive angles are measured counter-clockwise from the y-axis.

## Installation

The following lines of code install the `coco_rocket_lander` project package plus all additional dependencies:

In [ ]:
#%%capture
%load_ext autoreload
%autoreload 2

import numpy as np
np.set_printoptions(suppress=True, precision=3)


import matplotlib.pyplot as plt
from numpy.typing import NDArray

from select_deepc.base_controllers import RandomInputGenerator, UniformInputGenerator, RandomWalkInputGenerator
from select_deepc.deepc_controller import *
from select_deepc.deepc_utils import *
from select_deepc.deepc_rocket_utils import *

from itertools import product

import pandas as pd
import pickle

plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        "font.sans-serif": "Times",
        "font.size": 10,
    }
)

PAPER_COL_WIDTH_IN = (6.75 - 0.25) / 2
PAPER_CONTENT_WIDTH_IN = 6.75

color_iso = "salmon"
color_l1 = "chocolate"
color_rand = "orchid"

## Data acquisition

We let the rocket fly using random inputs. In order to not crash, we use the existing pid controller to stabilize the system if we deviate too far from the initial position. After successfully acquiring the data, we will attempt land the rocket.

### Function definitions

In [ ]:
def run_data_collection_experiment(env, controller, seed=0):
  obs, info = env.reset(seed=seed)  # specify a random seed for consistency

  obs_init = obs[:2].copy()

  simulation_iteration = 0

  state_trajectory_data_acquisition = []
  input_trajectory_data_acquisition = []


  landing_position = env.get_landing_position()

  # simulate
  while True:
    controller_obs = obs.copy()

    if simulation_iteration < 1200:
        controller_obs[0] = obs[0] - obs_init[0]
        controller_obs[1] = obs[1] - obs_init[1]

    else:
        controller_obs[0] = obs[0] - landing_position[0]
        controller_obs[1] = obs[1] - landing_position[1]

    # get action
    try:
        action = controller.compute_action(
          controller_obs,
          np.zeros(8),
          False if simulation_iteration < 1000 else True
        )
    except:
        action = controller.compute_action(controller_obs, np.zeros(8))

    next_obs, _, done, truncated, info = env.step(action)
    
    if not np.any(obs[6:]):
        state_trajectory_data_acquisition.append(next_obs.copy())

        input_trajectory_data_acquisition.append(action)



    # check if simulation ended
    if done or truncated:
      print(f"info:\n{info}")
      break

    # update observation
    obs = next_obs
    simulation_iteration += 1

  env.close()  # video saved at this step

  # Drop last two columns as they are not relevant for the state propagation
  state_trajectory_data_acquisition = np.array(state_trajectory_data_acquisition)[:,:6]
  input_trajectory_data_acquisition = np.array(input_trajectory_data_acquisition)
  print(state_trajectory_data_acquisition.shape)

  return state_trajectory_data_acquisition, input_trajectory_data_acquisition

In [ ]:
def collect_random_data(mode):
    if mode not in ["iid", "random_walk", "random_walk_2"]:
        assert ValueError(f"Invalid value: {mode}. Options are: 'iid' or 'random_walk'")

    rng = np.random.default_rng(42)
    for experiment_idx in range(100):
        state_filename = f"data/dataset_{mode}/{experiment_idx}_random_data_state.csv"
        input_filename = f"data/dataset_{mode}/{experiment_idx}_random_data_input.csv"

        env = get_simulator_environment(
            None,
            f"mt/data_collection_{mode}",
            f"{experiment_idx}_experiment",
            100,
            random_initial_position=True,
        )
        if mode == "iid":
            max_thrust = rng.uniform(0, 1)
            state_traj, input_traj = run_data_collection_experiment(
                env, UniformInputGenerator([0, -1, -1], [max_thrust, 1, 1], seed=experiment_idx), experiment_idx
            )
        elif mode =="random_walk":
            state_traj, input_traj = run_data_collection_experiment(
                env, RandomWalkInputGenerator([0, -1, -1], [max_thrust, 1, 1], [0.35, 0, 0], experiment_idx), experiment_idx
            )
        elif mode =="random_walk_2":
            state_traj, input_traj = run_data_collection_experiment(
                env, RandomWalkInputGenerator([0, -1, -1], [max_thrust, 1, 1], [0.35, 0, 0], experiment_idx, innovation=0.5), experiment_idx
            )
        np.savetxt(state_filename, state_traj, delimiter=",")
        np.savetxt(input_filename, input_traj, delimiter=",")



In [ ]:
def get_pid_data():
    """Loads the data which was collected with the pid if it exists, else runs the experiment"""
    pid_state_data_filename = "data/dataset_pid/pid_data_state.csv"
    pid_input_data_filename = "data/dataset_pid/pid_data_input.csv"
    try:
        state_trajectory_data_acquisition = np.loadtxt(pid_state_data_filename, delimiter=",")
        input_trajectory_data_acquisition = np.loadtxt(pid_input_data_filename, delimiter=",")
    except FileNotFoundError:
        class RandomInputGenerator(BaseController):
            """Pre-stabilized random input generator"""
            from coco_rocket_lander.algs import PID_Benchmark

            class PidWrapper(BaseController):
                def __init__(self):
                    engine_pid_params = [10, 0, 10]
                    engine_vector_pid_params = [0.085, 0.001, 10.55]
                    side_engine_pid_params = [5, 0, 6]

                    self._pid = self.PID_Benchmark(
                        engine_pid_params, engine_vector_pid_params, side_engine_pid_params
                    )

                def compute_action(self, state: NDArray, reference: NDArray) -> NDArray:
                    error = state - reference
                    return np.array(self._pid.pid_algorithm(error))

            def __init__(
                self, env, mu: Union[float, NDArray] = 0.0, cov: Union[float, NDArray] = 1.0
            ):
                self._pid = self.PidWrapper()
                self._noise_gen = lambda: np.random.multivariate_normal(
                    mu if isinstance(mu, (list, np.ndarray)) else mu * np.ones(3),
                    np.diag(cov) if isinstance(cov, (list, np.ndarray)) else cov * np.eye(3),
                    size=1,
                ).squeeze()

                main_engine_limits = env.cfg.main_engine_thrust_limits
                side_engine_limits = env.cfg.side_engine_thrust_limits
                nozzle_angle_limits = env.cfg.nozzle_angle_limits

                self._actuation_min = np.array(
                    [main_engine_limits[0], side_engine_limits[0], nozzle_angle_limits[0]]
                )
                self._actuation_max = np.array(
                    [main_engine_limits[1], side_engine_limits[1], nozzle_angle_limits[1]]
                )

            def compute_action(self, state, reference, pid_only=False):
                pid_action = self._pid.compute_action(state, reference)
                if pid_only or np.any(np.abs(state[4:6]) > np.array([0.1, 0.4])):
                    return pid_action.clip(self._actuation_min, self._actuation_max)

                noise = self._noise_gen().clip(self._actuation_min, self._actuation_max)
                return noise

            def normalize_input(self, action):
                return np.maximum(np.minimum(action, self._actuation_max), self._actuation_min)

        env = get_simulator_environment((0.5, 0.7, -0.2), "mt", "04_mf_data_acquisition", 2000)
        stabilized_random_inputs = RandomInputGenerator(env, [0.0, 0, 0], [2, 2, 2])
        state_trajectory_data_acquisition, input_trajectory_data_acquisition = run_data_collection_experiment(env, stabilized_random_inputs)
        show_video(0)

        np.savetxt(pid_state_data_filename, state_trajectory_data_acquisition, delimiter=",")

        np.savetxt(pid_input_data_filename, input_trajectory_data_acquisition, delimiter=",")

    return TrajectoryDataSet(
        [
            TrajectoryData(
                state_trajectory_data_acquisition,
                input_trajectory_data_acquisition,
            )
        ],
        "pid",
    )

### Data collection

In [ ]:
# Uncomment to generate data

# collect_random_data("iid")
# collect_random_data("random_walk")
# collect_random_data("random_walk_2")

In [ ]:
pid_trajectory = get_pid_data()
iid_trajectories = load_data_from_folder("data/dataset_iid", "iid")
random_walk_trajectories = load_data_from_folder("data/dataset_random_walk", "random_walk")
random_walk_trajectories_2 = load_data_from_folder("data/dataset_random_walk_2", "random_walk_2")
# prbs_trajectory = TrajectoryDataSet(
#     [TrajectoryData(
#         np.loadtxt("../DeePC-HUNT/examples/data/rocket_yd.csv", delimiter=","),
#         np.loadtxt("../DeePC-HUNT/examples/data/rocket_ud.csv", delimiter=","),
#     )],
#     "prbs",
# )

## DeePC

In [ ]:
def setup_DeePC(
    env: gym.Env, trajectory_data: TrajectoryDataSet
) -> DeePCControllerArgs:
    """"Set up the Data Driven Controller

  NOTE: Paramters in here can be adapted to tune the desired performance
  """

    mass, _ = env.get_mass_properties()
    gravity_comp_fraction = np.array([-env.cfg.gravity * mass / env.cfg.main_engine_thrust, 0, 0])

    p = 6 # sensor state size
    m = 3 # input size
    n = 75 # "estimated" system size
    T_past = 2
    T_fut = 30
    T_hankel= (m + 1) * (T_past + T_fut) + n - 1

    # x, y, dx, dy, theta, dtheta
    Q = np.diag([40,20,20,1,3000,300])
    R = np.diag([10, 10, 10])
    g_regularization_1 = 0
    g_regularization_pi =  5000
    slack_cost = 50000
    controller_costs = DeePCCost(Q, R, slack_cost, g_regularization_1, g_regularization_pi)

    A_constr_x = np.array([
    [-1, 0, 0, 0,  0, 0],
    [1,  0, 0, 0,  0, 0],
    [0, -1, 0, 0,  0, 0],
    [0,  1, 0, 0,  0, 0],
    [0,  0, 0, 0, -1, 0],
    [0,  0, 0, 0,  1, 0],
  ])
    b_constr_x = np.array([
    -0,
    env.cfg.width,
    -0,
    env.cfg.height,
    env.cfg.theta_limit,
    env.cfg.theta_limit,
  ])

    A_constr_u = np.array([
    [-1, 0, 0],
    [1, 0, 0],
    [0, -1, 0],
    [0, 1, 0],
    [0, 0, -1],
    [0, 0, 1],
  ])
    main_thruster_lower_lim = env.cfg.main_engine_thrust_limits[0] + 0.0
    b_constr_u = np.array([
    -main_thruster_lower_lim,
    env.cfg.main_engine_thrust_limits[1],
    -env.cfg.side_engine_thrust_limits[0],
    env.cfg.side_engine_thrust_limits[1],
    -env.cfg.nozzle_angle_limits[0],
    env.cfg.nozzle_angle_limits[1],
  ])

    controller_constraints = DeePCConstraints(A_constr_u, b_constr_u, A_constr_x, b_constr_x)

    return DeePCControllerArgs(
        trajectory_data,
        DeePCDims(
            T_past,
            T_fut,
            p,
            m,
        ),
        T_hankel,
        controller_costs,
        controller_constraints,
        gravity_comp_fraction,
        False,
    )

In [ ]:
env = get_simulator_environment((0.2, 0.9, 0.0), "mt", "rocket_streaming_deepc", 1000)
controller_args = setup_DeePC(
    env,
    random_walk_trajectories,
)
controller_args.controller_costs.regularizer_cost_g_1 = 8
controller_args.controller_costs.regularizer_cost_g_pi = 5000
H_u, H_y = HankelMatrixGenerator(controller_args.deepc_dims.T_past, controller_args.deepc_dims.T_fut).generate_hankel_matrices(controller_args.trajectory_data)
# deePC = StreamingDeePC(H_u[:,:1000], H_y[:,:1000], controller_args)
deePC = DataDrivenPredictiveController.create_from_data(
    controller_args,
    decompose_hankel_matrices=True,
)

(
    x_traj_deepc,
    u_traj_deepc,
    x_traj_deepc_predictions,
    u_traj_deepc_predictions,
    cost,
) = run_simulator(
    env,
    RocketControllerWrapper(deePC),
    # LandingScheduler(env),
    HoverSetPointScheduler(env),
    deepc_cost_accumulator=PerformanceAccumulatorWrapper(
        DeePCCostAccumulator(controller_args.controller_costs),
        IntegralSquareError(),
        IntegralAbsoluteError(),
    ),
)
print(cost)

In [ ]:
show_video(1, "mt")

In [ ]:
plot_trajectory(
    x_traj_deepc,
    x_traj_deepc_predictions,
    plot_heading=False,
)

## Windowed DeePC

In [ ]:
def test_sdeepc():
    initial_condition = {"x_ini":0.2, "y_ini": 0.9, "theta_ini": 0.0}
    env = get_simulator_environment(list(initial_condition.values()), "mt", "7_sdeepc_test", 1000)

    controller_args = setup_DeePC(
        env,
        iid_trajectories,
    )
    H_u, H_y = HankelMatrixGenerator(
        controller_args.deepc_dims.T_past,
        controller_args.deepc_dims.T_fut,
    ).generate_hankel_matrices(
        controller_args.trajectory_data, 
    )
    H_u = H_u[:, :1000]
    H_y = H_y[:, :1000]
    print(H_u.shape)
    print(H_y.shape)

    streaming_deepc = StreamingDeePC(
        H_u,
        H_y,
        controller_args,
    )

    (x_traj_deepc, u_traj_deepc, x_traj_deepc_predictions, u_traj_deepc_predictions, cost) = (
        run_simulator(
            env,
            RocketControllerWrapper(streaming_deepc),
            HoverSetPointScheduler(env), #LandingScheduler(env),
            seed=None,
            deepc_cost_accumulator=PerformanceAccumulatorWrapper(
                DeePCCostAccumulator(controller_args.controller_costs),
                IntegralSquareError(),
                IntegralAbsoluteError(),
            ),
        )
    )

    np.savetxt("data/rocket/wdeepc_traj.csv", x_traj_deepc)
    np.savetxt("data/rocket/wdeepc_traj_ol.csv",
            x_traj_deepc_predictions
            .transpose((0, 2, 1))
            .reshape(-1, x_traj_deepc_predictions.shape[1]),
        )
               
               
    # video displayed might be incorrect.
    show_video(3, "mt")

    plot_trajectory(x_traj_deepc, x_traj_deepc_predictions)

test_sdeepc()

## Select DeePC

In [ ]:
def run_weighted_deepc():
    perf_list = []
    datasets = [
        # random_walk_trajectories_2,
        # random_walk_trajectories,
        # random_walk_trajectories_2+pid_trajectory,
        # random_walk_trajectories+pid_trajectory,
        iid_trajectories,
    ]
    regularizer_weights = [8]
    num_iters = [1]
    # hankel_cols = [150, 200, 250, 300, 500, 1000, 2500, -1]
    hankel_cols = [500]
    for dataset, lambda_g, num_cols, num_iter in product(datasets, regularizer_weights, hankel_cols, num_iters):
        initial_condition = {"x_ini":0.2, "y_ini": 0.9, "theta_ini": 0.0}
        env = get_simulator_environment(list(initial_condition.values()), "mt", f"06_select_deepc_landing_iid-{num_cols}", 1000)
        controller_args = setup_DeePC(
            env,
            dataset,
        )
        controller_args.controller_costs.regularizer_cost_g_1 = lambda_g
        controller_args.controller_costs.regularizer_cost_g_pi = 5000

        num_cols = (
            num_cols
            if num_cols != -1
            else HankelMatrixGenerator(
                controller_args.deepc_dims.T_past,
                controller_args.deepc_dims.T_fut
            ).generate_hankel_matrices(controller_args.trajectory_data)[0].shape[-1]
        )

        selector = IsoMapEmbeddedDistances(
            controller_args.trajectory_data,
            controller_args.deepc_dims,
            use_scaler=False,
            # n_neighbors=4,
            # n_components=6 + controller_args.deepc_dims.T_fut * controller_args.deepc_dims.m
        )
        # selector = LkSelector()
        # selector = RandomSelector()

        x_traj_select_deepc, x_traj_select_deepc_preds, u_traj_select_deepc, cost = test_weighted_deepc(
            env, controller_args, num_cols, selector, num_iter
        )

        ol_preds_shape = x_traj_select_deepc_preds.shape
        trajectory_dir = os.path.join("data", "rocket", f"cl_{dataset.dataset_name}", str(num_cols))
        try:
            os.mkdir(trajectory_dir)
        except FileExistsError:
            pass

        np.savetxt(os.path.join(trajectory_dir, "state_traj.csv"), x_traj_select_deepc)
        # Save open loop x, y, vx, vy in that order
        np.savetxt(
            os.path.join(trajectory_dir, "state_traj_ol.csv"),
            x_traj_select_deepc_preds
            .transpose((0, 2, 1))
            .reshape(-1, ol_preds_shape[1]),
        )

        perf_list.append({
            "num_cols": num_cols,
            "num_iter": num_iter,
            "selector": selector.get_selector_name(),
            "n_neighbors": 4,
            "n_components": 6 + controller_args.deepc_dims.T_fut * controller_args.deepc_dims.m,
            **cost,
            **controller_args.get_logging(),
            **initial_condition,
        }
        )
        plot_trajectory(
            x_traj_select_deepc,
            x_traj_select_deepc_preds,
            plot_heading=False,
            filename=f"figures/{dataset.dataset_name}_dataset/s-deepc_{num_iter}_iter_{controller_args.controller_costs.print()}_col{num_cols}_{selector.get_selector_name()}.pdf",
        )
        show_video(2, "mt")
        print(f"{dataset.dataset_name}, {num_cols}")

    df = pd.DataFrame(perf_list)
    df.to_csv("data/costs_iso_new.csv", mode="a", index=False)


def test_weighted_deepc(env, controller_args, cols, selector, num_iter):
    wdeepc = SelectDeePC(
    controller_args,
    selector_callback = selector,
    num_hankel_cols = cols,
    n_iter=num_iter,
    )

    (
        x_traj_deepc, u_traj_deepc,
        x_traj_deepc_predictions, u_traj_deepc_predictions, cost
    ) = run_simulator(
        env,
        RocketControllerWrapper(wdeepc),
        # HoverSetPointScheduler(env),
        LandingScheduler(env),
        seed=None,
        deepc_cost_accumulator=PerformanceAccumulatorWrapper(
            DeePCCostAccumulator(controller_args.controller_costs),
            IntegralSquareError(),
            IntegralAbsoluteError(),
        ),
    )

    print(f"\n\n\n\n\n\n{cost}\n\n\n")
    return x_traj_deepc, x_traj_deepc_predictions, u_traj_deepc, cost


run_weighted_deepc()

## Utils stuff

In [ ]:
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

blue = plt.rcParams["axes.prop_cycle"].by_key()["color"][0]
orange = plt.rcParams["axes.prop_cycle"].by_key()["color"][1]
green = plt.rcParams["axes.prop_cycle"].by_key()["color"][2]

data_df = pd.read_csv("data/rocket/costs_iso_new.csv")

fig, axs = plt.subplots(3,1, figsize=(PAPER_COL_WIDTH_IN,4), sharex=True, sharey=False, dpi=400)
axs = axs.flatten()
axs_dict = {}

idx = -1
for ((dataset_name, theta_ini), group) in data_df[data_df["lambda_1"] == 8.0].groupby(["dataset_name", "theta_ini"]):
    if "2" in dataset_name:
        continue

    idx += 1

    group = group[group["cost"] != np.inf]
    if dataset_name == "random_walk":
        add_df = pd.DataFrame(
            {"num_cols": [5208], "cost": [10], "IAE": [.1], "ISE": [.1], "theta_ini": [0.0]},
        )
        group = pd.concat([group, add_df], ignore_index=True)
        # display(group)

    axs_dict[dataset_name] = axs[idx]

    axs[idx].semilogx(group["num_cols"], group["cost"], 'x-', label="deepc cost")
    axs[idx].semilogx(group["num_cols"], 100*group["IAE"], 'x-', label="IAE")
    axs[idx].semilogx(group["num_cols"], 100*group["ISE"], 'x-', label="ISE")

    axs[idx].set_title(f"costs on {dataset_name}".replace("_", " ").title())

data_df = pd.read_csv("data/rocket/costs_l1_new.csv") 

for ax in axs:
    ax.set_prop_cycle(None)

idx = -1
for (dataset_name, group) in data_df[data_df["lambda_1"] == 8.0].groupby("dataset_name"):
    if "2" in dataset_name:
        continue

    idx += 1
    for theta_ini, subgroup in group.groupby("theta_ini"):
        subgroup = subgroup[subgroup["cost"] != np.inf]
        if dataset_name == "random_walk":
            add_df = pd.DataFrame(
                {"num_cols": [2500], "cost": [10], "IAE": [.1], "ISE": [.1], "theta_ini": [0.0]},
            )
            subgroup = pd.concat([subgroup, add_df], ignore_index=True)

        axs_dict[dataset_name].semilogx(subgroup["num_cols"], subgroup["cost"], '^', label="deepc cost")
        axs_dict[dataset_name].semilogx(subgroup["num_cols"], 100*subgroup["IAE"], '^', label="IAE")
        axs_dict[dataset_name].semilogx(subgroup["num_cols"], 100*subgroup["ISE"], '^', label="ISE")

data_df = pd.read_csv("data/rocket/costs_random_new.csv")

for ax in axs:
    ax.set_prop_cycle(None)

idx = -1
for dataset_name, group in data_df[data_df["lambda_1"] == 8.0].groupby("dataset_name"):
    if "2" in dataset_name:
        continue

    idx += 1
    for theta_ini, subgroup in group.groupby("theta_ini"):
        subgroup = subgroup[subgroup["cost"] != np.inf]
        if dataset_name == "random_walk":
            add_df = pd.DataFrame(
                {
                    "num_cols": [2500],
                    "cost": [10],
                    "IAE": [0.1],
                    "ISE": [0.1],
                    "theta_ini": [0.0],
                },
            )
            subgroup = pd.concat([subgroup, add_df], ignore_index=True)

        axs_dict[dataset_name].semilogx(
            subgroup["num_cols"], subgroup["cost"], "+", label="deepc cost"
        )
        axs_dict[dataset_name].semilogx(
            subgroup["num_cols"], 100 * subgroup["IAE"], "+", label="IAE"
        )
        axs_dict[dataset_name].semilogx(
            subgroup["num_cols"], 100 * subgroup["ISE"], "+", label="ISE"
        )

custom_lines = [
    Line2D([0],[0], color='black', marker="x", linestyle="-"),
    Line2D([0],[0], color=blue, linestyle="-"),
    Line2D([0],[0], color='black', marker="^", linestyle=""),
    Line2D([0],[0], color=orange, linestyle="-"),
    Line2D([0],[0], color='black', marker="+", linestyle=""),
    Line2D([0],[0], color=green, linestyle="-"),
]
axs[-1].set_xlabel("Number of Cols")
fig.legend(
    custom_lines,
    ["$L_1$", "Cost", "Random", "IAE", "Isomap", "ISE"],
    loc="lower left",
    bbox_to_anchor=(0.05, -0.1),
    ncols=3,
)
for ax in axs:
    ax.grid()
axs[1].set_ylim(0, 3)
axs[1].set_ylabel("Cost")


expl = plt.imread("figures/explosion.png")
expl_img = OffsetImage(expl, zoom=0.05)
axs[1].add_artist(
    AnnotationBbox(
        expl_img,
        (3000, 2.75),
        frameon=False
    )
)


fig.tight_layout()

# Add labels and title
plt.savefig("figures/rows_vs_costs.pdf", bbox_inches="tight")

In [ ]:
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

data_df = pd.read_csv("data/rocket/costs_iso_new.csv")

fig, axs = plt.subplots(2,1, figsize=(PAPER_COL_WIDTH_IN,2.5), sharex=True, sharey=False, dpi=400)
axs = axs.flatten()
axs_dict = {}

idx = -1
for ((dataset_name, theta_ini), group) in data_df[data_df["lambda_1"] == 8.0].groupby(["dataset_name", "theta_ini"]):
    if "2" in dataset_name:
        continue

    if dataset_name == "random_walk_plus_pid":
        continue

    idx += 1

    group = group[group["cost"] != np.inf]
    if dataset_name == "random_walk":
        add_df = pd.DataFrame(
            {"num_cols": [5208], "cost": [10], "IAE": [.1], "ISE": [.1], "theta_ini": [0.0]},
        )
        group = pd.concat([group, add_df], ignore_index=True)

    axs_dict[dataset_name] = axs[idx]

    axs[idx].semilogx(group["num_cols"], group["cost"], ".--", color=color_iso, label="deepc cost")
    # axs[idx].semilogx(group["num_cols"], 100*group["IAE"], 'x-', label="IAE")
    # axs[idx].semilogx(group["num_cols"], 100*group["ISE"], 'x-', label="ISE")

    display_name = "Random Walk" if dataset_name == "random_walk" else "IID"
    axs[idx].set_title(f"{display_name} Data".replace("_", " "), fontsize="small")

data_df = pd.read_csv("data/rocket/costs_l1_new.csv") 

for ax in axs:
    ax.set_prop_cycle(None)

idx = -1
for (dataset_name, group) in data_df[data_df["lambda_1"] == 8.0].groupby("dataset_name"):
    if "2" in dataset_name:
        continue

    if dataset_name == "random_walk_plus_pid":
        continue

    idx += 1
    for theta_ini, subgroup in group.groupby("theta_ini"):
        subgroup = subgroup[subgroup["cost"] != np.inf]
        if dataset_name == "random_walk":
            add_df = pd.DataFrame(
                {"num_cols": [2500], "cost": [10], "IAE": [.1], "ISE": [.1], "theta_ini": [0.0]},
            )
            subgroup = pd.concat([subgroup, add_df], ignore_index=True)

        axs_dict[dataset_name].semilogx(subgroup["num_cols"], subgroup["cost"], ".--", color=color_l1, label="deepc cost")
        # axs_dict[dataset_name].semilogx(subgroup["num_cols"], 100*subgroup["IAE"], '^', label="IAE")
        # axs_dict[dataset_name].semilogx(subgroup["num_cols"], 100*subgroup["ISE"], '^', label="ISE")

data_df = pd.read_csv("data/rocket/costs_random_new.csv")

for ax in axs:
    ax.set_prop_cycle(None)

idx = -1
for dataset_name, group in data_df[data_df["lambda_1"] == 8.0].groupby("dataset_name"):
    if "2" in dataset_name:
        continue

    if dataset_name == "random_walk_plus_pid":
        continue

    idx += 1
    for theta_ini, subgroup in group.groupby("theta_ini"):
        subgroup = subgroup[subgroup["cost"] != np.inf]
        if dataset_name == "random_walk":
            add_df = pd.DataFrame(
                {
                    "num_cols": [2500],
                    "cost": [10],
                    "IAE": [0.1],
                    "ISE": [0.1],
                    "theta_ini": [0.0],
                },
            )
            subgroup = pd.concat([subgroup, add_df], ignore_index=True)

        axs_dict[dataset_name].semilogx(
            subgroup["num_cols"], subgroup["cost"], ".--", color=color_rand, label="deepc cost"
        )
        # axs_dict[dataset_name].semilogx(
        #     subgroup["num_cols"], 100 * subgroup["IAE"], "+", label="IAE"
        # )
        # axs_dict[dataset_name].semilogx(
        #     subgroup["num_cols"], 100 * subgroup["ISE"], "+", label="ISE"
        # )

# custom_lines = [
#     Line2D([0],[0], color='black', marker="x", linestyle="-"),
#     Line2D([0],[0], color=blue, linestyle="-"),
#     Line2D([0],[0], color='black', marker="^", linestyle=""),
#     Line2D([0],[0], color=orange, linestyle="-"),
#     Line2D([0],[0], color='black', marker="+", linestyle=""),
#     Line2D([0],[0], color=green, linestyle="-"),
# ]
# fig.legend(
#     custom_lines,
#     ["Iso", "Cost", "L1", "IAE", "Rand", "ISE"],
#     loc="lower left",
#     bbox_to_anchor=(0.05, -0.1),
#     ncols=3,
# )
axs[-1].set_xlabel(r"$N_\textrm{cols}$")

custom_lines = [
    Line2D([0],[0], color=color_rand, linestyle="--", marker="."),
    Line2D([0],[0], color=color_l1, linestyle="--", marker="."),
    Line2D([0],[0], color=color_iso, linestyle="--", marker="."),
]
fig.legend(
    custom_lines,
    ["Random", "$L_1$", "Isomap"],
    loc="lower left",
    title=r"\textbf{Selection Method}",
    bbox_to_anchor=(0.095, -0.16),
    ncols=3,
)
for ax in axs:
    ax.grid()

axs[1].set_ylim(None, 3)


expl = plt.imread("figures/explosion.png")
expl_img = OffsetImage(expl, zoom=0.05)
axs[1].add_artist(
    AnnotationBbox(
        expl_img,
        (2400, 2.75),
        frameon=False
    )
)
axs[1].add_artist(
    AnnotationBbox(
        expl_img,
        (2900, 2.75),
        frameon=False
    )
)
axs[1].add_artist(
    AnnotationBbox(
        expl_img,
        (1200, 2.75),
        frameon=False
    )
)

fig.suptitle("Closed-Loop Cost for the Rocket", x=0.55, y=0.9)

fig.tight_layout()

# Add labels and title
plt.savefig("figures/rows_vs_costs.pdf", bbox_inches="tight")

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
def plot_ol_traj(ol_arr:np.ndarray, axs, color):
    for i in range(0, ol_arr.shape[0], 6):
        plt.plot(ol_arr[i,:], ol_arr[i+1,:], linestyle="--", alpha=0.5, c=color)

blue = plt.rcParams["axes.prop_cycle"].by_key()["color"][0]
orange = plt.rcParams["axes.prop_cycle"].by_key()["color"][1]
green = plt.rcParams["axes.prop_cycle"].by_key()["color"][2]
fig, axs = plt.subplots(1,1, figsize=(PAPER_COL_WIDTH_IN, 2.75), dpi=600)

select_traj = np.loadtxt("data/rocket/cl_random_walk/500/state_traj.csv")
select_traj_ol = np.loadtxt("data/rocket/cl_random_walk/500/state_traj_ol.csv")
axs.plot(select_traj[:,0], select_traj[:,1], c=blue,zorder=10, label="Select-DeePC")
plot_ol_traj(select_traj_ol, axs, blue)

full_traj = np.loadtxt("data/rocket/cl_random_walk/5208/state_traj.csv")
full_traj_ol = np.loadtxt("data/rocket/cl_random_walk/5208/state_traj_ol.csv")
axs.plot(full_traj[:,0], full_traj[:,1], c=orange, label="Full Data DeePC")
plot_ol_traj(full_traj_ol, axs, orange)

select_traj_2 = np.loadtxt("data/rocket/wdeepc_traj.csv")
select_traj_ol_2 = np.loadtxt("data/rocket/wdeepc_traj_ol.csv")
axs.plot(select_traj_2[:,0], select_traj_2[:,1], c=green, label="Windowed DeePC")
plot_ol_traj(select_traj_ol_2, axs, green)


expl = plt.imread("figures/explosion.png")
expl_img = OffsetImage(expl, zoom=0.05)
rocket = plt.imread("figures/rocket.png")
rocket_img = OffsetImage(rocket, zoom=0.1)
axs.add_artist(
    AnnotationBbox(
        expl_img,
        (32, 13.5),
        frameon=False
    )
)
axs.add_artist(
    AnnotationBbox(
        expl_img,
        (32, 8),
        frameon=False
    )
)

axs.scatter(16.667, 12.48, c="red", marker="x", zorder=20, label="Setpoint")

# axs.set_box_aspect(1)
axs.legend()
axs.set_title("Rocket Closed-Loop Trajectory")
axs.set_xlabel("$x$ [m]")
axs.set_ylabel("$y$ [m]")
fig.tight_layout()
axs.set_xlim(0, 33)
# axs.set_aspect("equal")
fig.savefig("figures/closed_loop_comp.pdf")